In [1]:
import numpy as np
from astropy.io import fits
import cv2
import os
import math

In [2]:
def min_max(x, ax, ay):
    min_x = x.min()
    max_x = x.max()
    a = np.zeros((ay, ax))
    b = np.zeros((ay, ax))
    z = np.zeros((ay, ax))
    for i in range(ay):
        for j in range(ax):
            z[i][j] = (x[i][j] - min_x)/(max_x - min_x) * 255
            a[i][j],b[i][j] = math.modf(z[i][j])

    return a,b

In [3]:
def image_conv(image):
    i = fits.getdata(image)
    cv2.imwrite("test.tif",i.astype('uint16'))
    c = cv2.imread("test.tif",-1)

    return c

In [25]:
# FITS画像ファイルのリストを作成
# image_files = ["../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491939184_b.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491942076_v.fit",
#                "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491945018_w.fit",
#                  "../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2491947926_p.fit"]  # すべての画像ファイルを含むリスト

image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489428266_b.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489432123_v.fit",
               "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489436012_w.fit",
                 "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489439885_p.fit"]

# image_files = ["../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490234689_b.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490238562_v.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490242451_w.fit",
#                "../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2490246307_p.fit"]

# image_files = ["../preprocess_original/test/after_ref/cor1_st_2489428266_b.fit",
#                "../preprocess_original/test/after_ref/cor1_st_2489432123_v.fit",
#                "../preprocess_original/test/after_ref/cor1_st_2489436012_w.fit",
#                "../preprocess_original/test/after_ref/cor1_st_2489439885_p.fit"]

# image_files = ["../preprocess/test/before_ref/cor1_st_2489428266_b.fit",
#                "../preprocess//test/before_ref/cor1_st_2489432123_v.fit",
#                "../preprocess/test/before_ref/cor1_st_2489436012_w.fit",
#                "../preprocess/test/before_ref/cor1_st_2489439885_p.fit"]

# 出力ディレクトリとファイル名のプレフィックスを指定
home_dir = os.path.expanduser("~")
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_directory_op/")
file_prefix = 'registered_image_'  # ファイル名のプレフィックスを指定

hdulist = fits.open(image_files[2])  
hdu = hdulist[0]
data = hdu.data
header = hdu.header

name_prefix = []
# for i,name in enumerate(image_files):
#     parts = name.split(".f")
#     part = parts[0].split("/test/")
#     par = part[1].split("_ref/")
#     p = f"{par[1]}_reg_"
#     name_prefix.append(p)

for i,name in enumerate(image_files):
    parts = name.split(".f")
    part = parts[0].split("/data/")
    par = part[1].split("/st")
    p = f"{par[1]}_reg_"
    name_prefix.append(p)
# name = image_files[0]
# parts = name.split(".f")
# part = parts[0].split("/data/")
# par = part[1].split("/st")
# p = f"{par[0]}"
# name_prefix.append(p)

# 最初の画像を基準画像として読み込み
images = []
#少数部分の保存
float_images = []
f = []
images_uint16 = []
t = []
for i,filename in enumerate(image_files):
    print(filename)
    images_uint16.append(image_conv(filename))
    f.append(fits.getdata(filename))
    hdu = fits.open(filename)[0]
    t.append(hdu.header["EXP_0"])
    # t.append(hdu.header["TEXP"])
    a,b = min_max(f[i], f[i].shape[1], f[i].shape[0])
    if i == 2:
      reference_image = np.uint8(b)
    images.append(np.uint8(b))
    float_images.append(a)
  
print(t)


../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489428266_b.fit
../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489432123_v.fit
../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489436012_w.fit
../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051025/st_2489439885_p.fit
[0.131, 0.087, 0.0655, 0.174]


In [26]:
output_directory = os.path.join(home_dir, "Documents/after_graduate2/output_perspective/")
registered_images = []
registered_png = []


# 基準画像から他の画像への変換行列を計算し、変換を適用
for i,image_file in enumerate(images):
    
    # 画像の特徴点を検出
    akaze = cv2.AKAZE_create()
    keypoints1, descriptors1 = akaze.detectAndCompute(reference_image, None)
    keypoints2, descriptors2 = akaze.detectAndCompute(image_file, None)
    
    # 特徴点マッチングと変換行列の計算
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(descriptors2, descriptors1, k = 2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append([m])
    
    #適切なキーポイントを選択
    src_pts = np.float32([keypoints2[m[0].queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints1[m[0].trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    M, inliers = cv2.estimateAffine2D(src_pts, dst_pts)
    
    # 変換行列を適用して画像を基準画像の座標系に変換
    registered_image = cv2.warpAffine(images_uint16[i], M, (image_file.shape[1], image_file.shape[0]), flags=cv2.INTER_NEAREST)
    registered_png= cv2.warpAffine(image_file, M, (image_file.shape[1], image_file.shape[0]), flags=cv2.INTER_NEAREST)
    registered_image[(registered_image <= 1000) | (registered_image >= 5000)] = 0
    registered_images.append(registered_image)

for i in range(len(registered_images)):
    output_jpgname = f"{output_directory}{name_prefix[i]}.png"
    output_filename = f"{output_directory}{name_prefix[i]}.fits"
    cv2.imwrite(output_jpgname, registered_png)
    hdu = fits.PrimaryHDU(registered_images[i])
    hdu.header['TEXP']=t[i]
    hdulist = fits.HDUList([hdu])
    hdulist.writeto(output_filename, overwrite=True)



In [142]:
col_img = []
col_img = cv2.merge((registered_images[0],registered_images[1],registered_images[2]))
name = image_files[0]
parts = name.split(".f")
part = parts[0].split("/data/")
par = part[1].split("/st")
col_name = "_color"
p = f"{par[0]}{col_name}"
name_prefix.append(p)
output_coljpgname = f"{output_directory}{name_prefix[4]}_2.jpg"
cv2.imwrite(output_coljpgname, col_img)

True

In [159]:
affine_matrix = M

# アフィン変換行列から回転の角度、平行移動、拡大倍率を取得
angle = np.arctan2(affine_matrix[1, 0], affine_matrix[0, 0])
translation_x = affine_matrix[0, 2]
translation_y = affine_matrix[1, 2]
scale_x = affine_matrix[0, 0]
scale_y = affine_matrix[1, 1]

# 角度を度数法に変換
angle_degrees = np.degrees(angle)

cx = translation_x / np.cos(angle)
cy = translation_y / np.sin(angle)

# 結果
print("回転角度（度数法）:", angle_degrees)
print("X方向の平行移動:", translation_x)
print("Y方向の平行移動:", translation_y)
print("X方向の拡大倍率:", scale_x)
print("Y方向の拡大倍率:", scale_y)
print("回転中心のX座標:", cx)
print("回転中心のY座標:", cy)

回転角度（度数法）: 0.10421876184939545
X方向の平行移動: -1.9579941654023187
Y方向の平行移動: 1.778319978824197
X方向の拡大倍率: 0.9942943214010063
Y方向の拡大倍率: 1.0024314931226048
回転中心のX座標: -1.9579974045335997
回転中心のY座標: 977.6578016120237
